# Get-started (Community version)

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- A python environment

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create and develop a new promptflow batch run
- Evaluate the batch run with a evaluation flow
- Deploy the flow to a http endpoint.

**Motivations** - This guide will walk you through the main user journey of prompt flow code-first experience. You will learn how to create and develop your first prompt flow, test and evaluate it, then deploy it to production.

## Step 0: Install promptflow sdk

In [ ]:
!pip install promptflow-sdk[builtins] --extra-index-url https://azuremlsdktestpypi.azureedge.net/promptflow/

## Step 1: Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

In this notebook, we will use flow `web-classification` which uses connection `azure_open_ai_connection` inside, we need to set up the connection if we haven't added it before. After created, it's stored in local db and can be used in any flow.

In [ ]:
import promptflow as pf
from promptflow.sdk.entities import AzureOpenAIConnection

# client can help manage your runs and connections.
client = pf.PFClient()

In [ ]:
try:
    conn_name = "azure_open_ai_connection"
    conn = client.connections.get(name=conn_name)
    print("using existing connection")
except:
    connection = AzureOpenAIConnection(
        name=conn_name,
        api_key="<test_key>",
        api_base="<test_base>",
        api_type="azure",
        api_version="<test_version>",
    )

    conn = client.connections.create_or_update(connection)
    print("successfully created connection")

# TODO pretty print
print(conn)

## Step 2: Create and develop a new batch run

`web-classification` is a flow demonstrating multi-class classification with LLM. Given an url, it will classify the url into one web category with just a few shots, simple summarization and classification prompts.

### Set flow path and batch run input data

In [ ]:
flow = "../../flows/standard/web-classification" # path to the flow directory
data="../../flows/standard/web-classification/data.jsonl"  # path to the data file

### Create batch run

In [ ]:
# create batch run with default variant
run = pf.run_batch(
    flow=flow,
    data=data
)

In [ ]:
pf.stream(run)

In [ ]:
pf.show_details(run)

In [ ]:
pf.visualize([run])

## Step 3: Evaluate your bulk flow run
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which use Python or LLM etc., to calculate metrics like accuracy, relevance score.

In this notebook, we use `classification-accuracy-eval` flow to evaluate. This is a flow illustrating how to evaluate the performance of a classification system. It involves comparing each prediction to the groundtruth and assigns a "Correct" or "Incorrect" grade, and aggregating the results to produce metrics such as accuracy, which reflects how good the system is at classifying the data.

In [ ]:
eval_flow = "../../flows/evaluation/classification-accuracy-eval"

eval_run = pf.run_eval(
    flow=eval_flow,
    data="../../flows/standard/web-classification/data.jsonl",  # path to the data file
    batch_run=run,  # use the batch run as the variant
    inputs_mapping={"groundtruth": "${data.answer}","prediction": "${variant.outputs.category}"},  # map the url field from the data to the url input of the flow
)

In [ ]:
pf.stream(eval_run)

In [ ]:
pf.show_details(eval_run)

In [ ]:
pf.show_metrics(eval_run)

In [ ]:
pf.visualize([run, eval_run])

In [ ]:
# use the first variant of the summarize_text_content node.
# if "node_variant" is not specified in "run_batch" function, the default variant will be used.
node_variant="${summarize_text_content.variant_1}"
variant_run = pf.run_batch(
    flow=flow,
    data=data,
    node_variant=node_variant,  
)

# TODO run evaluation twice and compare the metrics


## Step 4: Deploy the flow (WIP)